In [1]:
import pandas as pd
import random
import numpy as np
import tensorflow as tf

In [2]:
data=pd.read_csv("../HW-2-Machine-reasoning/dataset1.csv")
data.head(10)

,x,f(x)
0,-0.66,18.37
1,2.99,5.00
2,2.65,5.12
3,-1.72,27.31
4,4.33,6.76
5,1.94,6.13
6,-1.02,21.17
7,0.05,13.73
8,-1.77,27.78
9,1.06,8.76


In [3]:
data_x = data["x"]
data_x.head(10)

0   -0.66
1    2.99
2    2.65
3   -1.72
4    4.33
5    1.94
6   -1.02
7    0.05
8   -1.77
9    1.06
Name: x, dtype: float64

In [4]:
data_y = data["f(x)"]
data_y.head(10)

0    18.37
1     5.00
2     5.12
3    27.31
4     6.76
5     6.13
6    21.17
7    13.73
8    27.78
9     8.76
Name: f(x), dtype: float64

In [5]:
F = ['+','-','*','/']

In [6]:
T = ['x']+ list(range(-5,6))

In [7]:
print(T)

['x', -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]


In [8]:
class Node():
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
    def insert_left(self, child):
        if (self.left == None):
            self.left = child
        else:
            child.left = self.left
            self.left = child
    def insert_right(self, child):
        if(self.right == None):
            self.right = child
        else:
            child.right = self.right
            self.right = child
def print2D(root, space) :

    if (root == None) :
        return
    # Increase distance between levels
    space += 10

    print2D(root.right, space)

    # Print current node after space
    print()
    for i in range(10, space):
        print(end = " ")
    print(root.val)

    print2D(root.left, space)

root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)
root.right.left = Node(6)
root.right.right = Node(7)
root.right.right.insert_left(Node(20))
print2D(root,0)


                    7

                              20

          3

                    6

1

                    5

          2

                    4
